#### BMCR exploration

In [10]:
!aws s3 ls --no-sign-request s3://brainminds-marmoset-connectivity/

                           PRE BMCR_v02/


In [14]:
!aws s3 ls --no-sign-request s3://brainminds-marmoset-connectivity/BMCR_v02/

                           PRE BMCR_STPT_template/
                           PRE BMCR_core_data/
2025-06-04 16:26:39        444 Changelog.txt
2025-06-04 16:26:39       3398 LICENSE.txt


In [12]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = 'brainminds-marmoset-connectivity'

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket)

def insert(d, keys, info):
    """Helper to insert file info into a nested dict, based on S3 key split."""
    for k in keys[:-1]:
        d = d.setdefault(k, {})
    d[keys[-1]] = info

tree = {}

for page in pages:
    for obj in page.get('Contents', []):
        path_parts = obj['Key'].split('/')
        # Remove empty names (can occur with trailing '/')
        path_parts = [p for p in path_parts if p]
        info = {
            'Size': obj['Size'],
            'LastModified': str(obj['LastModified']),
            'StorageClass': obj.get('StorageClass', 'STANDARD')
        }
        insert(tree, path_parts, info)

# Example: show the top-level structure
import json
print(json.dumps(tree, indent=2)[:2000])  # only first 2k chars for preview

# Save the full structure to a JSON file
with open('BMCR_SkibbeData_AWS_structure_objects.json', 'w') as f:
    json.dump(tree, f, indent=2)

{
  "BMCR_v02": {
    "BMCR_STPT_template": {
      "ANTs_trafos": {
        "BMA_v2": {
          "BMA_2_STPT.h5": {
            "Size": 2728334600,
            "LastModified": "2025-06-04 07:14:13+00:00",
            "StorageClass": "INTELLIGENT_TIERING"
          },
          "BMA_2_STPT_affine.h5": {
            "Size": 12792,
            "LastModified": "2025-06-04 07:14:13+00:00",
            "StorageClass": "STANDARD"
          },
          "BMA_2_STPT_merged.nii.gz": {
            "Size": 204720026,
            "LastModified": "2025-06-04 07:14:13+00:00",
            "StorageClass": "INTELLIGENT_TIERING"
          },
          "Readme.txt": {
            "Size": 475,
            "LastModified": "2025-06-04 07:14:13+00:00",
            "StorageClass": "STANDARD"
          },
          "STPT_2_BMA.h5": {
            "Size": 2728334600,
            "LastModified": "2025-06-04 07:14:13+00:00",
            "StorageClass": "INTELLIGENT_TIERING"
          },
          "STPT_2_BMA_affi

In [11]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import tempfile
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import numpy as np

# S3 bucket and key
bucket = 'brainminds-marmoset-connectivity'
#key = 'BMCR_v02/BMCR_core_data/Processed_tracer_data/A11-R04_0058-TT75/3D_stacks/anterograde_cell_density.nii.gz'  # <-- replace with actual key
#key = 'BMCR_v02/BMCR_core_data/Processed_tracer_data/ProM-R01_0060-TT37/3D_stacks/streamline_density.nii.gz'
key = 'BMCR_v02/BMCR_core_data/Processed_tracer_data/ProM-R01_0060-TT37/3D_stacks/anterograde_tracer_positive_voxels.nii.gz'
# Set up S3 client for public bucket (no credentials needed)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Download S3 object to named temporary file
with tempfile.NamedTemporaryFile(suffix='.nii.gz') as tmpfile:
    s3.download_fileobj(bucket, key, tmpfile)
    tmpfile.flush()  # Ensure all data is written

    img = nib.load(tmpfile.name)
    data = img.get_fdata()

    slider = IntSlider(min=0, max=data.shape[2]-1, step=1, value=0)

    def show_slice(slice_idx):
        plt.imshow(data[:, :, slice_idx].T, cmap="viridis", origin="lower")
        plt.title(f"Slice {slice_idx}")
        plt.axis('off')
        plt.show()

    interactive_plot = interact(show_slice, slice_idx=slider)


interactive(children=(IntSlider(value=0, description='slice_idx', max=489), Output()), _dom_classes=('widget-i…

In [6]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

resp = s3.list_objects_v2(Bucket='brainminds-marmoset-connectivity')
for obj in resp.get('Contents', []):
    print(obj['Key'], obj['Size'])

BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/BMA_2_STPT.h5 2728334600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/BMA_2_STPT_affine.h5 12792
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/BMA_2_STPT_merged.nii.gz 204720026
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/Readme.txt 475
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/STPT_2_BMA.h5 2728334600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/STPT_2_BMA_affine.h5 12792
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/STPT_2_BMA_merged.nii.gz 232557774
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/merged_trafos.txt 654
BMCR_v02/BMCR_STPT_template/ANTs_trafos/MBCA/MBCA_2_STPT.h5 341054600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/MBCA/STPT_2_MBCA.h5 341054600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/MBM_v2/MBM_2_STD.h5 341054600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/MBM_v2/STD_2_MBM.h5 341054600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/MBM_v3/MBM_2_STPT.h5 341054600
BMCR_v02/BMCR_STPT_template/ANTs_trafos/MBM_v3/Readme.txt 748


In [7]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = 'brainminds-marmoset-connectivity'

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket)

file_dict = {}

for page in pages:
    for obj in page.get('Contents', []):
        file_dict[obj['Key']] = {
            'Size': obj['Size'],
            'LastModified': obj['LastModified'],
            'StorageClass': obj.get('StorageClass', 'STANDARD')
        }

# Example usage:
for k, v in list(file_dict.items())[:5]:
    print(k, v)

BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/BMA_2_STPT.h5 {'Size': 2728334600, 'LastModified': datetime.datetime(2025, 6, 4, 7, 14, 13, tzinfo=tzutc()), 'StorageClass': 'INTELLIGENT_TIERING'}
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/BMA_2_STPT_affine.h5 {'Size': 12792, 'LastModified': datetime.datetime(2025, 6, 4, 7, 14, 13, tzinfo=tzutc()), 'StorageClass': 'STANDARD'}
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/BMA_2_STPT_merged.nii.gz {'Size': 204720026, 'LastModified': datetime.datetime(2025, 6, 4, 7, 14, 13, tzinfo=tzutc()), 'StorageClass': 'INTELLIGENT_TIERING'}
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/Readme.txt {'Size': 475, 'LastModified': datetime.datetime(2025, 6, 4, 7, 14, 13, tzinfo=tzutc()), 'StorageClass': 'STANDARD'}
BMCR_v02/BMCR_STPT_template/ANTs_trafos/BMA_v2/STPT_2_BMA.h5 {'Size': 2728334600, 'LastModified': datetime.datetime(2025, 6, 4, 7, 14, 13, tzinfo=tzutc()), 'StorageClass': 'INTELLIGENT_TIERING'}
